In [1]:
#https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-dataset
!rm -rf sample_data
!echo "Downloading ⬇️⬇️⬇️"
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
!echo "Data downloaded ⬇️"
!wget https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12_2.zip
!echo "Annotations downloaded ⤵️"

!echo "Untaring 🤐"
!tar zxf mpii_human_pose_v1.tar.gz
!echo "Untared 🤐"

!echo "Unziping 🤐"
!unzip mpii_human_pose_v1_u12_2.zip
!echo "Unziped 🤐"

!echo "Data ready ✅"

--2025-04-19 07:54:09--  https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1.tar.gz
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12088943206 (11G) [application/x-gzip]
Saving to: ‘mpii_human_pose_v1.tar.gz’

mpii_human_pose_v1. 100%[===================>]  11.26G  29.7MB/s    in 6m 47s  

2025-04-19 08:00:56 (28.4 MB/s) - ‘mpii_human_pose_v1.tar.gz’ saved [12088943206/12088943206]

Data downloaded ⬇️
--2025-04-19 08:00:56--  https://datasets.d2.mpi-inf.mpg.de/andriluka14cvpr/mpii_human_pose_v1_u12_2.zip
Resolving datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)... 139.19.206.177
Connecting to datasets.d2.mpi-inf.mpg.de (datasets.d2.mpi-inf.mpg.de)|139.19.206.177|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12340483 (12M) [application/zip]


In [27]:
# Data annotations described here:
# https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-dataset/download

import scipy.io
import numpy as np

mat_file_path = 'mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'
release_data = scipy.io.loadmat(mat_file_path)['RELEASE']
print(f"Fields inside 'RELEASE' structured array: {release_data.dtype.names}")
annolist = release_data['annolist'][0, 0]
print(f"Shape of 'annolist': {annolist.shape}")


first_image_annots = annolist[0, 0] # Or maybe annolist[0] depending on shape
print("\nInspecti----", first_image_annots.dtype.names)
print(first_image_annots['image'][0][0][0][0])

act = release_data['act'][0, 0]

print(f"act ::: {act}")

filenames_map = {}


for i in range(annolist.shape[1]):  # Iterate through the second dimension of annolist
    image_annot = annolist[0, i]  # Access the element using index i in the second dimension
    filename = image_annot['image'][0][0][0][0]  # Extract the filename

    filenames_map[i] = filename


    if i < 5:
      print(f"Filename for index {i}: {filename}")



Fields inside 'RELEASE' structured array: ('annolist', 'img_train', 'version', 'single_person', 'act', 'video_list')
Shape of 'annolist': (1, 24987)

Inspecti---- ('image', 'annorect', 'frame_sec', 'vididx')
037454012.jpg
act ::: [[(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 ...
 [(array(['transportation'], dtype='<U14'), array(['pushing car'], dtype='<U11'), array([[972]], dtype=uint16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]]
Filename for index 0: 037454012.jpg
Filename for index 1: 095071431.jpg
Filename for index 2: 073199394.jpg
Filename for index 3: 059865848.jpg
Filename for index 4: 015601864.jpg


In [43]:
act = release_data['act'][0, 0]
anotations_map = {}

for i, item in enumerate(act):
    try:
        act_name = item[0][0][0] if len(item[0]) > 0 and len(item[0][0]) > 0 else None
        cat_name = item[0][1][0] if len(item[0]) > 1 and len(item[0][1]) > 0 else None
        #act_id = item[0][2][0][0] if len(item[0]) > 2 and len(item[0][2]) > 0 else None
    except IndexError:
        print("IndexError encountered. Skipping this item.")
        continue

    anotations_map[i] = (str(filenames_map[i]), str(act_name), str(cat_name).split(", "))



# Ziurim ka turim. todo
for i in anotations_map:
    filename, act_name, cat_name = anotations_map[i]
    if i > 100:
       break
    print(f"I: {i}. Filename {filename} Act Name: {act_name}, Cat Name: {cat_name}")
    #print(f"{type(filename)}, Act Name: {type(act_name)}, Cat Name: {type(cat_name)}")

I: 0. Filename 037454012.jpg Act Name: None, Cat Name: ['None']
I: 1. Filename 095071431.jpg Act Name: None, Cat Name: ['None']
I: 2. Filename 073199394.jpg Act Name: None, Cat Name: ['None']
I: 3. Filename 059865848.jpg Act Name: None, Cat Name: ['None']
I: 4. Filename 015601864.jpg Act Name: sports, Cat Name: ['curling']
I: 5. Filename 015599452.jpg Act Name: sports, Cat Name: ['curling']
I: 6. Filename 005808361.jpg Act Name: sports, Cat Name: ['curling']
I: 7. Filename 086617615.jpg Act Name: sports, Cat Name: ['curling']
I: 8. Filename 060111501.jpg Act Name: sports, Cat Name: ['curling']
I: 9. Filename 070807258.jpg Act Name: sports, Cat Name: ['curling']
I: 10. Filename 002058449.jpg Act Name: sports, Cat Name: ['curling']
I: 11. Filename 096990985.jpg Act Name: None, Cat Name: ['None']
I: 12. Filename 083244264.jpg Act Name: None, Cat Name: ['None']
I: 13. Filename 021233911.jpg Act Name: inactivity quiet/light, Cat Name: ['sitting quietly']
I: 14. Filename 018182497.jpg Act Na

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import scipy.io as sio
import os
from PIL import Image
import numpy as np
import PIL

# Define the custom dataset class
class MPIIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #(np.str_('061172075.jpg'), np.str_('water activities'), np.str_('swimming, sidestroke, general'), np.uint16(261))
        print(f"------------------------{self.data[idx]}")
        image_path, act_name, cat_name, act_id = self.data[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format

        if self.transform:
            image = self.transform(image)

        # Return the image and label (adjust label processing as needed)
        return image, act_name

# Create a list to store the dataset (image path, label) pairs
dataset = anotations_map

# Split the dataset into train, validation, and test sets
# You can adjust the split ratios as needed
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(dataset) * train_ratio)
val_size = int(len(dataset) * val_ratio)
test_size = len(dataset) - train_size - val_size

train_dataset = {}
val_dataset = {}
test_dataset = {}

keys = list(anotations_map.keys())  # Get all keys from the dictionary

train_size = int(len(anotations_map) * train_ratio)
val_size = int(len(anotations_map) * val_ratio)

# Assign elements to datasets based on key index
for i, key in enumerate(keys):
    if i < train_size:
        train_dataset[key] = anotations_map[key]
    elif i < train_size + val_size:
        val_dataset[key] = anotations_map[key]
    else:
        test_dataset[key] = anotations_map[key]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

transform = transforms.Compose([
    transforms.Resize(256),  # Resize the images
    transforms.CenterCrop(224),  # Crop the images
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

train_dataset = MPIIDataset(train_dataset, transform=transform)
val_dataset = MPIIDataset(val_dataset, transform=transform)
test_dataset = MPIIDataset(test_dataset, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Train dataset size: 19989
Validation dataset size: 2498
Test dataset size: 2500


In [21]:
# prompt: generate me pytorch model that trains on beforementioned dataloders

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define the model architecture
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 256), # Adjust input size based on image dimensions after conv layers.
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Assuming you have num_classes
num_classes = len(set([v[1] for v in anotations_map.values() if v[1]]))

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Use appropriate loss based on your task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10  # Adjust the number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(train_loader): # Use tqdm for progress bar
        images, labels = images.to(device), labels.to(device)  # Move data to the device
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Validation/Testing
model.eval()
# ... (Add your validation or testing loop here)


  0%|          | 0/625 [00:00<?, ?it/s]

------------------------('072967841.jpg', 'winter activities', 'skiing, downhill', '291')


FileNotFoundError: [Errno 2] No such file or directory: '072967841.jpg'